#### 测试tf的安装

In [7]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))
a = tf.constant(10)
b = tf.constant(32)
print(sess.run(a + b))
sess.close()

b'Hello, TensorFlow!'
42


#### mnist为例测试

In [6]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("E:/DeepLearning/datasets/MNIST/MNIST_data", one_hot=True)
print("Train data size: ", mnist.train.images.shape, mnist.train.labels.shape) #mnist.train.num_examples,
print("Validate data size: ", mnist.validation.images.shape, mnist.validation.labels.shape)
print("Testing data size: ", mnist.test.images.shape, mnist.test.labels.shape)

sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32,[None,784])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)#softmax(Wx+b)
y_ = tf.placeholder(tf.float32, [None, 10])

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))#每一个batch求均值

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
tf.global_variables_initializer().run()

for i in range(1000):
    batch_size = 100
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    train_step.run({x: batch_xs, y_: batch_ys})

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval({x: mnist.test.images, y_: mnist.test.labels}))

Extracting E:/DeepLearning/datasets/MNIST/MNIST_data\train-images-idx3-ubyte.gz
Extracting E:/DeepLearning/datasets/MNIST/MNIST_data\train-labels-idx1-ubyte.gz
Extracting E:/DeepLearning/datasets/MNIST/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting E:/DeepLearning/datasets/MNIST/MNIST_data\t10k-labels-idx1-ubyte.gz
Train data size:  (55000, 784) (55000, 10)
Validate data size:  (5000, 784) (5000, 10)
Testing data size:  (10000, 784) (10000, 10)
0.9191


#### 使用卷积结构优化

In [8]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("E:/DeepLearning/datasets/MNIST/MNIST_data", one_hot=True)

sess = tf.InteractiveSession()

#初始化权重和偏置
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)# 截断的正态分布噪声
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
x_image = tf.reshape(x, [-1, 28, 28, 1])

#第一层卷积
W_conv1 = weight_variable([5, 5 , 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)#28*28
h_pool1 = max_pool_2x2(h_conv1)         #14*14

#二层
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)#14*14
h_pool2 = max_pool_2x2(h_conv2)#7*7    1/4的原图像大小

#全连接层
W_fc1 = weight_variable([7 * 7 * 64, 1024]) #tensor尺寸7 * 7 * 64，全连接层隐含节点数1024
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

#
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#
tf.global_variables_initializer().run()
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob:1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_:batch[1], keep_prob:0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0}))

Extracting E:/DeepLearning/datasets/MNIST/MNIST_data\train-images-idx3-ubyte.gz
Extracting E:/DeepLearning/datasets/MNIST/MNIST_data\train-labels-idx1-ubyte.gz
Extracting E:/DeepLearning/datasets/MNIST/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting E:/DeepLearning/datasets/MNIST/MNIST_data\t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.16
step 100, training accuracy 0.76
step 200, training accuracy 0.86
step 300, training accuracy 0.96
step 400, training accuracy 0.86
step 500, training accuracy 0.9
step 600, training accuracy 0.98
step 700, training accuracy 0.92
step 800, training accuracy 0.96
step 900, training accuracy 0.96
step 1000, training accuracy 0.98
step 1100, training accuracy 0.98
step 1200, training accuracy 0.96
step 1300, training accuracy 0.98
step 1400, training accuracy 0.96
step 1500, training accuracy 0.96
step 1600, training accuracy 0.96
step 1700, training accuracy 0.98
step 1800, training accuracy 1
step 1900, training accuracy 0.98
step 2000, trainin